In [73]:
import pandas as pd
import numpy as np

In [74]:
from get_all_DD_features import *

In [75]:
def preprocess_conversation_columns(df):
	# remove all special characters from df
	df.columns = df.columns.str.replace('[^A-Za-z0-9_]', '', regex=True)
	
	# If data is grouped by batch/round, add a conversation num
	if {'batch_num', 'round_num'}.issubset(df.columns):
		df['conversation_num'] = df.groupby(['batch_num', 'round_num']).ngroup()
		df = df[df.columns.tolist()[-1:] + df.columns.tolist()[0:-1]] # make the new column first

	return(df)

In [76]:
csop = preprocess_conversation_columns(pd.read_csv('../data/raw_data/csopII_conversations_withblanks.csv'))
vect = pd.read_csv('../embeddings/csopII_conversations_withblanks.csv')
csop['message_embedding'] = conv_to_float_arr(vect['message_embedding'].to_frame())

In [77]:
from get_all_DD_features import *

In [78]:
csop_chunked = assign_chunk_nums(csop, 5)
csop_chunked

,conversation_num,batch_num,vis_img,int_verb,ort_img,rep_man,soc_pers,team_size,difficulty,score,duration,efficiency,speaker_nickname,message,timestamp,message_embedding,chunk_num
727,25PJSrbtr8mkHTvcw,1,Mixed,High,Low,High,Mixed,2,Easy [Corresponds to 'Hard' in PNAS],604.0,559.0,1.080501,NiBqtamR7LYPByEoz,I think the next combination above 500 that wo...,2021-03-19T18:20:28.277Z,"[0.0190779697149992, -0.010986153967678547, -0...",0
728,25PJSrbtr8mkHTvcw,1,Mixed,High,Low,High,Mixed,2,Easy [Corresponds to 'Hard' in PNAS],604.0,559.0,1.080501,NiBqtamR7LYPByEoz,I think 504 would still be ok?,2021-03-19T18:22:33.265Z,"[-0.02542596124112606, 0.08265342563390732, -0...",0
729,25PJSrbtr8mkHTvcw,1,Mixed,High,Low,High,Mixed,2,Easy [Corresponds to 'Hard' in PNAS],604.0,559.0,1.080501,NiBqtamR7LYPByEoz,604-100,2021-03-19T18:22:41.278Z,"[-0.03982061892747879, 0.08185563236474991, -0...",0
730,25PJSrbtr8mkHTvcw,1,Mixed,High,Low,High,Mixed,2,Easy [Corresponds to 'Hard' in PNAS],604.0,559.0,1.080501,hF7iWkQvsomPnXY7S,sure,2021-03-19T18:22:50.937Z,"[-0.1395024061203003, -0.03979051858186722, -0...",1
3099,25PJSrbtr8mkHTvcw,1,Mixed,High,Low,High,Mixed,2,Hard [Corresponds to 'Super Hard' in PNAS],729.0,179.0,4.072626,NULL_SPEAKER,NaN,NULL_TIME,"[-0.0642978772521019, -0.028240203857421875, 0...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,zwtfREfmLE7LzyW7n,1,High,Mixed,Mixed,Mixed,High,4,Easy [Corresponds to 'Hard' in PNAS],537.0,294.0,1.826531,ttp99Mg4k27H4G4g2,you have ruined it all,2021-03-11T18:33:02.259Z,"[-0.014426716603338718, 0.14449119567871094, 0...",1
254,zwtfREfmLE7LzyW7n,1,High,Mixed,Mixed,Mixed,High,4,Easy [Corresponds to 'Hard' in PNAS],537.0,294.0,1.826531,eiZmgSepzvcWzXZnq,IT WAS FINE BEFORE,2021-03-11T18:33:11.705Z,"[-0.026309503242373466, -0.029762502759695053,...",2
255,zwtfREfmLE7LzyW7n,1,High,Mixed,Mixed,Mixed,High,4,Easy [Corresponds to 'Hard' in PNAS],537.0,294.0,1.826531,eiZmgSepzvcWzXZnq,AB HAVE TO BE NEIGHBORS,2021-03-11T18:33:26.463Z,"[0.022271981462836266, -0.0574793741106987, -0...",2
256,zwtfREfmLE7LzyW7n,1,High,Mixed,Mixed,Mixed,High,4,Easy [Corresponds to 'Hard' in PNAS],537.0,294.0,1.826531,ReHqydvhHuWfF9wbc,can we please put it back,2021-03-11T18:33:29.370Z,"[-0.053512245416641235, 0.0669332817196846, 0....",3


In [79]:
#if one out of two chunks is NAN --> variance will be NAN
get_variance_in_DD(csop_chunked)

,conversation_num,variance_in_DD
0,25PJSrbtr8mkHTvcw,NaN
1,28LFTFoBTha9WeCxT,NaN
2,28coGosspMQZSSsKK,NaN
3,2GyDrExReibZDZbHK,NaN
4,2PHmsZkuyuXMZcv7s,NaN
...,...,...
476,zRx7pqdbWDwduoYuM,0.002601
477,zXofkTp388eu67YxF,0.004885
478,zdBGjnPDQBJoyZqjh,0.006044
479,zoijP9Z4yNJ8sRgZq,NaN


In [80]:
mean_vec_speaker_chunks = pd.DataFrame(csop_chunked.groupby(['conversation_num', 'speaker_nickname', 'chunk_num']).message_embedding.apply(np.mean)).unstack('chunk_num').rename(columns={'message_embedding': 'mean_chunk_vec'})
mean_vec_speaker_chunks.columns = ["_c".join(col).strip() for col in mean_vec_speaker_chunks.columns.values]
mean_vec_speaker_chunks


mean_chunk_vec_c0   
conversation_num  speaker_nickname                                                       
25PJSrbtr8mkHTvcw NULL_SPEAKER                                                     NaN  \
                  NiBqtamR7LYPByEoz  [-0.015389536817868551, 0.05117430134365956, -...   
                  hF7iWkQvsomPnXY7S                                                NaN   
28LFTFoBTha9WeCxT NULL_SPEAKER       [-0.06429783999919891, -0.028240181505680084, ...   
28coGosspMQZSSsKK NULL_SPEAKER       [-0.06429783999919891, -0.028240181505680084, ...   
...                                                                                ...   
zwtfREfmLE7LzyW7n NULL_SPEAKER                                                     NaN   
                  ReHqydvhHuWfF9wbc                                                NaN   
                  bwnSEo9Y7pNyaTtwP  [0.08106686919927597, 0.11048600822687149, -0....   
                  eiZmgSepzvcWzXZnq  [0.027963388711214066, 0.06178921088576317, -0...   
                  ttp99Mg4k27H4G4g2  [-0.0795283168554306, 0.02962481416761875, 0.0...   

                                                                     mean_chunk_vec_c1   
conversation_num  speaker_nickname                                                       
25PJSrbtr8mkHTvcw NULL_SPEAKER       [-0.0642978772521019, -0.028240203857421875, 0...  \
                  NiBqtamR7LYPByEoz                                                NaN   
                  hF7iWkQvsomPnXY7S  [-0.1395024061203003, -0.03979051858186722, -0...   
28LFTFoBTha9WeCxT NULL_SPEAKER                                                     NaN   
28coGosspMQZSSsKK NULL_SPEAKER                                                     NaN   
...                                                                                ...   
zwtfREfmLE7LzyW7n NULL_SPEAKER                                                     NaN   
                  ReHqydvhHuWfF9wbc  [-0.026309503242373466, -0.029762502759695053,...   
                  bwnSEo9Y7pNyaTtwP                                                NaN   
                  eiZmgSepzvcWzXZnq                                                NaN   
                  ttp99Mg4k27H4G4g2  [-0.014426716603338718, 0.14449119567871094, 0...   

                                                                     mean_chunk_vec_c2   
conversation_num  speaker_nickname                                                       
25PJSrbtr8mkHTvcw NULL_SPEAKER                                                     NaN  \
                  NiBqtamR7LYPByEoz                                                NaN   
                  hF7iWkQvsomPnXY7S                                                NaN   
28LFTFoBTha9WeCxT NULL_SPEAKER                                                     NaN   
28coGosspMQZSSsKK NULL_SPEAKER                                                     NaN   
...                                                                                ...   
zwtfREfmLE7LzyW7n NULL_SPEAKER                                                     NaN   
                  ReHqydvhHuWfF9wbc                                                NaN   
                  bwnSEo9Y7pNyaTtwP                                                NaN   
                  eiZmgSepzvcWzXZnq  [-0.0020187608897686005, -0.043620938435196877...   
                  ttp99Mg4k27H4G4g2                                                NaN   

                                                                     mean_chunk_vec_c3   
conversation_num  speaker_nickname                                                       
25PJSrbtr8mkHTvcw NULL_SPEAKER                                                     NaN  \
                  NiBqtamR7LYPByEoz                                                NaN   
                  hF7iWkQvsomPnXY7S                                                NaN   
28LFTFoBTha9WeCxT NULL_SPEAKER                                                     NaN   
28coGosspMQZSSsK

In [81]:
num_chunks = 5
inter_chunk_range = [ [] for i in range(num_chunks - 1)]

for conv_idx, row in mean_vec_speaker_chunks.iterrows():
    for index in range(num_chunks - 1):
        tpl = [row['mean_chunk_vec_c' + str(index)], row['mean_chunk_vec_c'+ str(index+1)]]
        value = 0
        if (pd.isnull(tpl)).any():
            value = np.nan
        else:
            value = 1 - get_cosine_similarity(tpl)[0]
        inter_chunk_range[index].append(value)

index = []
for i in range(num_chunks - 1):
    index.append("c" + str(i) + "_c" + str(i + 1))
range_df = pd.DataFrame(inter_chunk_range, index=index).T
range_df['conversation_num'] = mean_vec_speaker_chunks.reset_index()['conversation_num']
range_df = range_df.set_index('conversation_num')

In [82]:
range_df.loc['zXofkTp388eu67YxF'].T

conversation_num,zXofkTp388eu67YxF,zXofkTp388eu67YxF,zXofkTp388eu67YxF,zXofkTp388eu67YxF,zXofkTp388eu67YxF,zXofkTp388eu67YxF,zXofkTp388eu67YxF,zXofkTp388eu67YxF
c0_c1,0.869169,0.379993,0.700863,0.785056,NaN,NaN,0.839221,NaN
c1_c2,0.754307,0.425573,0.880453,0.830621,NaN,NaN,NaN,NaN
c2_c3,0.628539,0.514887,0.708946,0.550860,NaN,NaN,NaN,NaN
c3_c4,0.566964,0.819543,0.549594,0.539897,NaN,NaN,0.682000,NaN


In [83]:
var_disc_range = range_df.groupby('conversation_num').apply(lambda x: (np.nanmean(x, axis=0)).sum()).to_frame().rename(columns={0:'var_disc_range'})
var_disc_range


/var/folders/q6/rj1_hcnn4l1czvx3phrf8kbw0000gn/T/ipykernel_52146/362978384.py:1: RuntimeWarning: Mean of empty slice
  var_disc_range = range_df.groupby('conversation_num').apply(lambda x: (np.nanmean(x, axis=0)).sum()).to_frame().rename(columns={0:'var_disc_range'})


,var_disc_range
conversation_num,
25PJSrbtr8mkHTvcw,NaN
28LFTFoBTha9WeCxT,NaN
28coGosspMQZSSsKK,NaN
2GyDrExReibZDZbHK,NaN
2PHmsZkuyuXMZcv7s,NaN
...,...
zRx7pqdbWDwduoYuM,NaN
zXofkTp388eu67YxF,2.670006
zdBGjnPDQBJoyZqjh,NaN
